# Named Entity Recognition system

In this lab, we are going to build a named entity recogniser similar to Lample et al,. (2016), and will train and evaluate your system on the benchmark CoNLL 2003 English datasets. You will get part of the code required to build the system, and you are required to fill two code blocks. Hints will be provided to guide you through.
In total, you will be given one notebook file (*.ipynb), three JSON files (*.json) and one embedding file (*.txt.filtered).
- this notebook is the main script you are going to work on
- The train/test/dev.conll03.json documents are the training, testing and development set that will be used for training and evaluating the model, which are converted from the original BIO files to JSON for your convenience.
- glove.6B.100d.txt.filtered is pre-trained 100-dimensional Glove word embeddings. The original file is large, so we‘ve removed all the words that do not appear in the datasets to make it much smaller.

First let's downlowd the datasets and the embedding file.

In [1]:
!wget 'https://github.com/juntaoy/ECS7001_LAB_DATASETS/raw/refs/heads/main/NER_data.zip'
!unzip NER_data.zip -x __MACOSX/*

--2025-04-21 21:26:16--  https://github.com/juntaoy/ECS7001_LAB_DATASETS/raw/refs/heads/main/NER_data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/juntaoy/ECS7001_LAB_DATASETS/refs/heads/main/NER_data.zip [following]
--2025-04-21 21:26:17--  https://raw.githubusercontent.com/juntaoy/ECS7001_LAB_DATASETS/refs/heads/main/NER_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8526560 (8.1M) [application/zip]
Saving to: ‘NER_data.zip’

NER_data.zip        100%[===================>]   8.13M  --.-KB/s    in 0.1s    

2025-04-21 21:26:18 (80.1 MB/s) - ‘NER_data.zip’ saved [8526560/8526560]

The script contains all the elements needed for a simple named entity recognition system. Here we use the IO format and treat the flat NER as the sequence labelling task. Unlike BIO format, the IO format does not differentiate the start of the entity and the remaining tokens hence simpler. The following is an example encoded in IO format:

| Token | Label |
| :-------------- | ----------- |
| Steve | PER |
| Jobs | PER |
| co-founded | O |
| Apple | ORG |
| in | O |
| his | O |
| father’s | O |
| garage | O |
| in | O |
| California | LOC |
| . | O |

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
# from transformers import AdamW

import numpy as np
import json, time, collections, random

## Define the network parameters and paths
Here we hardcoded them. For a real system, usually, the parameters will be stored in a configuration file, as there will be many of them. You might find this familiar as most of the deep learning models will need those parameters specified.

In [3]:
#The path to the pre-trained word embeddings:
embedding_path = 'glove.6B.100d.txt.ner.filtered'

#The dimension of the pre-trained embeddings, which in our case is 100:
embedding_size = 100

#A dropout rate for word embeddings (this is usually larger than the hidden dropout rate for the neural networks).
embedding_dropout_rate = 0.5

#The size of the hidden layer, including both GRU and feedforward NN
hidden_size = 50

#The number of hidden layers used for the feedforward NN
ffnn_layer = 2

#The dropout rate for the hidden layers of GRU and feedforward NN
hidden_dropout_rate = 0.2

#The model also keeps a record of the NER labels and maps them into numbers so that can be used for prediction.
ner_labels = ['O', 'PER', 'ORG', 'LOC', 'MISC']
ner_labels_mappings = {l:i for i,l in enumerate(ner_labels)}

## Load pre-trained word embeddings
After setting the network parameters, we load the pre-trained word embeddings from the given location by calling the load_embeddings method:

In [4]:
def load_embeddings():
    print("Loading word embeddings from {}...".format(embedding_path))
    embeddings = collections.defaultdict(lambda: np.zeros(embedding_size))
    with open(embedding_path, 'r', encoding='utf-8') as f:
        for line in f:
            splitter = line.find(' ')
            emb = np.fromstring(line[splitter + 1:], np.float32, sep=' ')
            assert len(emb) == embedding_size
            word = line[:splitter]
            embeddings[word] = emb
    print("Finished loading word embeddings")
    return embeddings

In [5]:
embedding_dict = load_embeddings()

Loading word embeddings from glove.6B.100d.txt.ner.filtered...
Finished loading word embeddings


## The `BiGRUFFNN` class builds the model for our task.
It first creates the input of the model. The `word_embeddings` contains the word embeddings of the tokens in the sentences and are stored as batches of sentences padded to the same length. The shape for `word_embeddings` is `[batch_size, max_sent_length, embedding_size]`. In our case, we use a batch size of 100 sentences. We then apply dropout to the embeddings; this helps us to reduce overfitting.

After that, there is your first task: to create a 2-layer bidirectional GRU and a multi-layer feedforward neural network to compute categorical scores for each token in the sentences. GRUs are similar to LSTMs but with fewer gates, hence more efficient; the two models are usually interchangeable. In this task, the GRUs take the `word_embeddings` as the input and output `word_output` for individual tokens in the sentences. More detailed instructions will be mentioned in the later section.

After we get the `word_output` we can simply feed it to a multi-layer feedforward neural network to compute the ner_scores. The ner_scores represent the probability of a token belonging to a NER label (one of O, PER, ORG, LOC, MISC) predicted by the system. Hence the last dimension of the `ner_scores` should equal the size of the ner labels. In our case, the shape of `ner_scores` is `[batch_size, max_sent_length, num_of_ner_labels]`.


In [6]:
# Define the BiGRUFFNN class
class BiGRUFFNN(nn.Module):
    """
    Task 1: Create a 2-layer bidirectional GRU and a multi-layer FFNN
    """
    def __init__(self,
                 embedding_size,
                 embedding_dropout_rate,
                 hidden_size,
                 ffnn_layer,
                 hidden_dropout_rate,
                 num_labels):
        super(BiGRUFFNN, self).__init__()

        # Dropout for embeddings
        self.embedding_dropout = nn.Dropout(embedding_dropout_rate)

        # 2-layer bidirectional GRU
        self.gru = nn.GRU(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            dropout=hidden_dropout_rate,
            batch_first=True
        )

        # Multi-layer FFNN with 2 hidden layers and output layer
        self.ffnn = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),  # Bidirectional GRU doubles the hidden size
            nn.ReLU(),
            nn.Dropout(hidden_dropout_rate),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(hidden_dropout_rate),
            nn.Linear(hidden_size, num_labels)
        )

    def forward(self, word_embeddings):
        """
        word_embeddings: Tensor of shape [batch_size, max_sent_length, embedding_size]
        Returns ner_scores: Tensor of shape [batch_size, max_sent_length, num_labels]
        """
        # Apply dropout to embeddings
        word_embeddings = self.embedding_dropout(word_embeddings)

        # Pass through GRU
        output, _ = self.gru(word_embeddings)  # output: [batch_size, max_sent_length, 2 * hidden_size]

        # Pass through FFNN to get NER scores
        ner_scores = self.ffnn(output)  # ner_scores: [batch_size, max_sent_length, num_labels]

        return ner_scores

In [7]:
ner_model = BiGRUFFNN(
        embedding_size=embedding_size,
        embedding_dropout_rate=embedding_dropout_rate,
        hidden_size=hidden_size,
        ffnn_layer=ffnn_layer,
        hidden_dropout_rate=hidden_dropout_rate,
        num_labels=len(ner_labels)
    )

# Print the model architecture
print(ner_model)

BiGRUFFNN(
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (gru): GRU(100, 50, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (ffnn): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=50, out_features=5, bias=True)
  )
)


## The `get_feed_dict_list()` method creates inputs for the NER model.
This method reads documents from the JSON files and returns a list of NumPy arrays, which are used as the input for the `ner_model` (`word_emb, word_ner_labels`). The method also returns the additional information (`gold_named_entities, sent_lengths`) which is used for evaluation.

Each of the JSON files contains only a single line, which includes two lists, one for sentences and one for gold named entities. The “`sentences`” field provides you with the tokenized sentences of the dataset and the “`ners`” element stores the indices and categories of gold named entities for each sentence.

We need to split the sentences into batches and create the associated gold IO labels, gold named entities and sentence lengths for the batches.

After creating the batches we store them in a `feed_dict_list` for further use.

In [8]:
def get_feed_dict_list(path, batch_size):
    feed_dict_list = []
    data_sets = json.loads(open(path).readlines()[0])
    sentences = data_sets['sentences']
    ners = data_sets['ners']

    for i in range(0, len(sentences), batch_size):
        batch_start, batch_end = i, min(i+batch_size, len(sentences))
        sent_lengths = [len(sent) for sent in sentences[batch_start:batch_end]]
        max_sent_length = max(sent_lengths)

        # Word embeddings
        word_emb = np.zeros([len(sent_lengths), max_sent_length, embedding_size], dtype=np.float32)
        for bi, sent in enumerate(sentences[batch_start:batch_end]):
            for j, word in enumerate(sent):
                word_emb[bi, j] = embedding_dict[word.lower()]

        # Word NER labels
        word_ner_labels = np.zeros([len(sent_lengths), max_sent_length], dtype=np.int64)
        gold_named_entities = set()
        for bi, ner in enumerate(ners[batch_start:batch_end]):
            for s, e, l in ner:
                l_id = ner_labels_mappings[l]
                gold_named_entities.add((bi, s, e, l_id))
                for j in range(s, e + 1):
                    word_ner_labels[bi, j] = l_id

        feed_dict_list.append((
            word_emb,
            word_ner_labels,
            gold_named_entities,
            sent_lengths
        ))

    return feed_dict_list

## The `batch_generator()` method returns a single batch for training.
The method takes the whole training set and shuffles the data. Then each time it generates one training batch in the format required by the `ner_model`.

In [9]:
def batch_generator(fd_list,device):
    random.shuffle(fd_list)
    for word_embeddings, word_ner_labels, _, _ in fd_list:
      yield torch.Tensor(word_embeddings).to(torch.float32).to(device), torch.Tensor(word_ner_labels).to(torch.long).to(device)


## The `eval()` method runs a test on the given dataset.
The method first creates a loop to iterate the datasets under evaluation to get predictions from the system.

However, the predictions are the `ner_scores` from the system and can’t be directly used to compute the entity level F1 score which we used to evaluate the NER systems. The second task, therefore, is to create the predicted named entities and to count the numbers of true positive, false negative and false positive entities. Those numbers can be later used to compute the precision, recall and F1 scores for the system. Again we will give detailed instructions of the second task in later sections.


In [10]:
# Evaluation function
def eval(eval_fd_list, device):
    tp, fn, fp = 0, 0, 0
    with torch.no_grad():
        for word_embeddings, _, gold, sent_lens in eval_fd_list:
            word_embeddings = torch.Tensor(word_embeddings).to(device)
            predictions = ner_model(word_embeddings)
            predictions = predictions.detach().cpu().numpy()

            """
            Task 2: Form predicted named entities from IO labels
            """
            # Get predicted labels
            predicted_labels = np.argmax(predictions, axis=-1)  # Shape: [batch_size, max_sent_length]
            predicted_entities = set()

            # Iterate over each sentence in the batch
            for bi in range(predictions.shape[0]):
                sent_len = sent_lens[bi]
                prev_label = 0
                start = -1

                # Iterate over tokens in the sentence
                for j in range(sent_len):
                    label = predicted_labels[bi, j]
                    if label != 0:  # Not 'O'
                        if label != prev_label:
                            if prev_label != 0:
                                # End previous entity
                                predicted_entities.add((bi, start, j-1, prev_label))
                            start = j
                            prev_label = label
                        # Else continue the entity
                    else:
                        if prev_label != 0:
                            # End previous entity
                            predicted_entities.add((bi, start, j-1, prev_label))
                        start = -1
                        prev_label = 0

                # Check if an entity ends at the sentence boundary
                if prev_label != 0:
                    predicted_entities.add((bi, start, sent_len-1, prev_label))

            # Compute TP, FN, FP
            tp_set = predicted_entities & gold
            tp += len(tp_set)
            fn += len(gold - predicted_entities)
            fp += len(predicted_entities - gold)

    # Compute precision, recall, and F1
    p = 0.0 if tp == 0 else tp * 1.0 / (tp + fp)
    r = 0.0 if tp == 0 else tp * 1.0 / (tp + fn)
    f = 0.0 if tp == 0 else 2 * p * r / (p + r)
    print("F1 : {:.2f}%".format(f * 100))
    print("Precision: {:.2f}%".format(p * 100))
    print("Recall: {:.2f}%".format(r * 100))

## The `time_used()` method outputs the time differences between the current time and the input time.
It is always good practice to record the time usage of an individual process, so you always know which part is most expensive to run.

In [11]:
def time_used(start_time):
  curr_time = time.time()
  used_time = curr_time - start_time
  m = used_time // 60
  s = used_time - 60 * m
  return "%d m %d s" % (m, s)

## The `train()` method oversees the training process.
It first loads the training/development/testing data into batches; Then training the model by going through all the training batches a number of times. It also outputs the time usage of the training. After each epoch, the model evaluates the development set. Normally, the model will be written to the disk if a better dev score is obtained. Here we didn’t do that to simplify the code for lab use. After finishing all the training epochs, it evaluates the final test set.


In [12]:
def train(train_path, dev_path, test_path, epochs,batch_size=100, device="cpu"):
    ner_model.to(device)
    ner_model.train()

    train_fd_list = get_feed_dict_list(train_path, batch_size)
    print("Load {} training batches from {}".format(len(train_fd_list), train_path))

    dev_fd_list = get_feed_dict_list(dev_path,batch_size)
    print("Load {} dev batches from {}".format(len(dev_fd_list), dev_path))

    test_fd_list = get_feed_dict_list(test_path,batch_size)
    print("Load {} test batches from {}".format(len(test_fd_list), test_path))

    # We'll use Adam and CrossEntropy for classification
    optimizer = optim.Adam(ner_model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss(reduction='mean')

    start_time = time.time()
    for epoch in range(epochs):
        print("\nStarting training epoch {}/{}".format(epoch + 1, epochs))
        epoch_time = time.time()

        # --- Example custom training loop ---
        # Shuffle training data
        generator = batch_generator(train_fd_list, device)
        # For each batch
        for inputs, labels in generator:
            # Zero out the gradients
            optimizer.zero_grad()
            # Forward pass
            logits = ner_model(inputs)
            # Reshape for cross entropy
            # logits: [batch_size, seq_len, num_labels]
            # labels: [batch_size, seq_len]
            # => Flatten them
            batch_size_curr, seq_len_curr, num_labels = logits.shape
            logits_2d = logits.view(-1, num_labels)
            labels_1d = labels.view(-1)

            loss = loss_fn(logits_2d, labels_1d)
            # Backprop
            loss.backward()
            optimizer.step()
        # --- End custom training loop ---

        print("Time used for epoch {}: {}".format(epoch + 1, time_used(epoch_time)))
        dev_time = time.time()
        print("Evaluating on dev set after epoch {}/{}:".format(epoch + 1, epochs))
        eval(dev_fd_list, device)
        print("Time used for evaluate on dev set: {}".format(time_used(dev_time)))

    print("\nTraining finished!")
    print("Time used for training: {}".format(time_used(start_time)))

    print("\nEvaluating on test set:")
    test_time = time.time()
    eval(test_fd_list, device)
    print("Time used for evaluate on test set: {}".format(time_used(test_time)))

## Task 1: Create a bidirectional GRU and Multi-layer FFNN
For task 1 we will be working on the `__init__` and `forward` method. The task is to create a bidirectional GRU to encode the sentences from both directions, which provides context information to the NER system.

The variables you will be using are:
- The dropout rate of the hidden layers: `hidden_dropout_rate`
- The word embeddings of the document: `word_embeddings`
- The size of the hidden layers: `hidden_size`

Firstly, you need to create a two-layer bidirectional GRU (BiGRUs) by using `nn.GRU`. The BiGRUs need to return the output for all the tokens in the sentences, not just the final one. The output of the BiGRUs should be called `word_output`.

**Note**: The default setting will return output you needed from the GRUs. And don’t forget to apply the `dropout`.

After we got the `word_output` from the BiGRUs, we can apply a multi-layer feedforward neural network to compute the ner_scores for each token. You are required to create an FFNN that contains 2 hidden layers and an output layer. The outputs of the FFNN are `ner_scores`. Here are the requirements:
1. The hidden layers need to have a size of `hidden_size`
2. You need to apply dropout to all the hidden layers (but not the output layer)
3. The outputs are called `ner_scores` with a shape of `[batch_size, max_sent_lengths, num_of_ner_labels]`

**Tips**:

Each hidden layer of the FFNN is a simple `nn.Linear` with a relu activation function (`nn.ReLU`). Layers are simply stacked together the output of the previous layer is the input for the next layer. To apply the dropout you can simply use the `nn.Dropout` layer. The output layer is slightly different since it will have an output size of the number of NER labels.

## Task 2: Form the predicted named entities.
The second task is to form the predicted named entities. For this, you will need to complete
the `eval()` method.

To complete this task you will need first to get the predicted labels from the predictions; you can do this by applying the `np.argmax()` method, the method returns the position of maximum values for the given axis. This will give you the labels for each token, e.g. 0 means label “O” and 1 means “PER” etc. Once you get the label you will need to go through all the sentences and tokens to create the predicted named entities by grouping the continuous tokens with the same NER label. The format of the predicted named entities is (`sid, start, end, ner_label`) where `sid` is the sentence’s position in the batch (start from 0), e.g. the ninth sentence of the batch will have a `sid` of 8. The `start` and `end` are the indices of the span in the sentence (both inclusive), the `ner_label` are the numeric labels according to `ner_labels_mappings`. Do ignore the label “O” for the evaluation, as “O” means non-named entity. Take the following sentence as an example:

|Token|Predction
|:---|:---
|Steve |1
|Jobs| 1
|co-founded| 0
|Apple |2
|in |0
|his |0
|father’s| 0
|garage| 0
|in| 0
|California| 3
|.|0

The numbers in the second column correspond to the actual NER labels `{“O”:0, “PER”:1, “ORG”:2, “LOC”:3, “MISC”:4}` defined in the `ner_labels_mappings` variable. For the above example, suppose the `sid` is 8. The system outputs should be `(8, 0, 1,1 ), (8, 3, 3, 2)` and `(8, 9, 9, 3)`. Those predicted entities will be used to compute the true positive, false positive and false negative numbers when compared with the gold named entities.

**Hint**: since the sentences are padded to the same length, you need to use the `sent_lens` to guide your process by only exploring the actual sentences and not the paddings.

## Run your code.
If you finished all the tasks, congratulations! you’ve made a NER system. The next step is to train your system on the data provided. Please make sure all the files are located in the same folder and then use the next cell to start your training. If all your code is correct your training should finish in a few minutes on a Google Colab with GPU (the time might take longer for local machines) and have F1 scores above 70% on both dev and test sets.

In [13]:
#the paths of the files
train_path = 'train.conll03.json'
dev_path = 'dev.conll03.json'
test_path = 'test.conll03.json'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train(train_path, dev_path, test_path, 25, device=device)

Load 141 training batches from train.conll03.json
Load 33 dev batches from dev.conll03.json
Load 35 test batches from test.conll03.json

Starting training epoch 1/25
Time used for epoch 1: 0 m 1 s
Evaluating on dev set after epoch 1/25:
F1 : 14.26%
Precision: 26.00%
Recall: 9.83%
Time used for evaluate on dev set: 0 m 0 s

Starting training epoch 2/25
Time used for epoch 2: 0 m 0 s
Evaluating on dev set after epoch 2/25:
F1 : 37.34%
Precision: 45.75%
Recall: 31.54%
Time used for evaluate on dev set: 0 m 0 s

Starting training epoch 3/25
Time used for epoch 3: 0 m 0 s
Evaluating on dev set after epoch 3/25:
F1 : 49.43%
Precision: 55.00%
Recall: 44.88%
Time used for evaluate on dev set: 0 m 0 s

Starting training epoch 4/25
Time used for epoch 4: 0 m 0 s
Evaluating on dev set after epoch 4/25:
F1 : 57.57%
Precision: 67.13%
Recall: 50.39%
Time used for evaluate on dev set: 0 m 0 s

Starting training epoch 5/25
Time used for epoch 5: 0 m 0 s
Evaluating on dev set after epoch 5/25:
F1 : 60.